In [145]:
load("ckks.py")

In [146]:
N = 2**15  # Ring degree
n = 2**2  # Number of slots
L = 20  # Number of levels
q0 = 2**40  # Smallest modulus
delta = 2**33  # Scaling factor

# The grouping parameter for bootstrapping is s. It lies between 1 and
# log(n, 2). The smaller s, the faster the bootstrapping, but the more levels
# are consumed.
s = log(n, 2)

CKKS.config(N, n, L, q0, delta)
CKKS.key_gen()
CKKS.config_bootstrap(CKKS.sk, s)

Setting the scheme's parameters...
Generating matrices required for encoding and decoding...
The CKKS configuration is done!

Setting the scheme's key parameters...
Generating secret key...
Generating public key...
Generating evaluation key...
Estimated security: 2^(80.0).
The key generation is done!

Encoding relevant vectors as polynomials...
Generating matrices required for CoeffToSlot and SlotToCoeff...
Encoding these matrices as polynomials...
Generating missing switching keys...
The bootstrapping configuration is done!



In [ ]:
# Creating two random complex vectors

complex_vectors = [
    np.array([randint(-3, 3) + 1j * randint(-3, 3) for _ in range(n)])
    for _ in range(2)
]
complex_vectors

[array([-1.+2.j,  3.+3.j, -3.+1.j, -3.+1.j]),
 array([3.-2.j, 3.+2.j, 1.+0.j, 0.+3.j])]

In [ ]:
# Encoding as polynomials

plaintext_polys = [CKKS.encode(z) for z in complex_vectors]
plaintext_polys

[- 8589934592 - 8757871169X^4096 + 16703502750X^8192 - 5271241842X^12288 + 15032385536X^16384 + 1303209465X^20480 - 7592501250X^24576 + 7114258342X^28672 mod(2^700),
 15032385536 - 10061080633X^4096 + 3037000500X^8192 - 3486629326X^12288 + 6442450944X^16384 - 7454661704X^20480 - 12148002000X^24576 - 8417467806X^28672 mod(2^700)]

In [ ]:
# Encrypting

ciphertexts = [CKKS.enc_poly_with_sk(pt, CKKS.sk) for pt in plaintext_polys]
ciphertexts

[A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20),
 A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20)]

In [ ]:
# Checking correctness

for i in range(2):
    pt = ciphertexts[i].dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[-1.+2.j  3.+3.j -3.+1.j -3.+1.j]
[ 3.00000000e+00-2.00000000e+00j  3.00000000e+00+2.00000000e+00j
  1.00000000e+00+1.54110724e-10j -4.90465446e-11+3.00000000e+00j]


In [ ]:
# Homomorphic operations

ct_add = ciphertexts[0] + ciphertexts[1]
ct_mul = (
    ciphertexts[0] @ ciphertexts[1]
)  # Polynomial multiplication followed by a rescaling operation
print(ct_add)
print(ct_mul)

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20)
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^19 (level 19 out of 20)


In [ ]:
# Checking correctness

for ct in [ct_add, ct_mul]:
    pt = ct.dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[ 2.-1.04280362e-09j  6.+5.00000000e+00j -2.+1.00000000e+00j
 -3.+4.00000000e+00j]
[ 1. +8.j          3.+15.00000001j -3. +1.j         -3. -9.j        ]


In [ ]:
# Bootstrapping

ct = ciphertexts[0] % q0  # Project to the lowest level l = 0
print(ct)

ct_boot = ct.bootstrap(s)
print(ct_boot)

pt = ct_boot.dec_to_poly(CKKS.sk)
print(CKKS.decode(pt))

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^0 (level 0 out of 20)
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^2 (level 2 out of 20)
[-1.00022469+1.99496871j  2.99849642+2.99465042j -2.99850067+1.00077645j
 -2.99817893+1.00048786j]
